In [13]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, lpDot, value, LpStatus

# 读取预处理后的数据
data = pd.read_excel("预处理后的数据.xlsx")

# 确保数据中所有需要的列都有值
required_columns = ["作物名称", "平均价", "亩产量/斤", "种植成本/(元/亩)", "作物类型"]
if not all(col in data.columns for col in required_columns):
    raise ValueError("数据缺少必要的列")

# 提取相关数据
years = range(2024, 2031)
crops = data["作物名称"].unique()
plots = data["地块名称"].unique()

# 定义问题
prob1_1 = LpProblem("Problem_1_1", LpMaximize)
prob1_2 = LpProblem("Problem_1_2", LpMaximize)

# 定义变量
x1_1 = LpVariable.dicts("x1_1", [(i, j, t) for i in plots for j in crops for t in years], lowBound=0, cat='Continuous')
x1_2 = LpVariable.dicts("x1_2", [(i, j, t) for i in plots for j in crops for t in years], lowBound=0, cat='Continuous')

# 设定目标函数
for j in crops:
    for t in years:
        crop_data = data.loc[data["作物名称"] == j].iloc[0]
        profit_per_unit = crop_data["平均价"] * crop_data["亩产量/斤"] - crop_data["种植成本/(元/亩)"]
        prob1_1 += lpDot([x1_1[i, j, t] for i in plots], profit_per_unit)

        total_production = lpDot([x1_2[i, j, t] for i in plots], crop_data["亩产量/斤"])
        excess = total_production - crop_data["预计销售量"]
        prob1_2 += profit_per_unit * total_production - 0.5 * crop_data["平均价"] * excess

# 添加约束条件
# 面积约束
for i in plots:
    for t in years:
        prob1_1 += lpSum([x1_1[i, j, t] for j in crops]) <= data.loc[data["地块名称"] == i, "种植面积/亩"].values[0]
        prob1_2 += lpSum([x1_2[i, j, t] for j in crops]) <= data.loc[data["地块名称"] == i, "种植面积/亩"].values[0]

# 水稻与蔬菜种植面积约束
for i in plots:
    if data.loc[data["地块名称"] == i, "地块类型"].values[0] == "水浇地":
        for t in years:
            prob1_1 += lpSum([x1_1[i, j, t] for j in crops if j == "水稻" or data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("蔬菜")]) <= 1
            prob1_2 += lpSum([x1_2[i, j, t] for j in crops if j == "水稻" or data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("蔬菜")]) <= 1
    else:
        for t in years:
            prob1_1 += lpSum([x1_1[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食") and j!= "水稻"]) <= 1
            prob1_2 += lpSum([x1_2[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食") and j!= "水稻"]) <= 1

# 轮作约束
for i in plots:
    for t in years:
        if t <= 2026:
            prob1_1 += lpSum([x1_1[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食（豆类）")]) >= 1
            prob1_2 += lpSum([x1_2[i, j, t] for j in crops if data.loc[data["作物名称"] == j, "作物类型"].values[0].startswith("粮食（豆类）")]) >= 1

# 求解问题
prob1_1.solve()
prob1_2.solve()

# 提取结果
result1_1 = pd.DataFrame([(i, j, t, value(x1_1[i, j, t])) for i in plots for j in crops for t in years if value(x1_1[i, j, t]) > 0], columns=["地块名称", "作物名称", "年份", "种植面积/亩"])
result1_2 = pd.DataFrame([(i, j, t, value(x1_2[i, j, t])) for i in plots for j in crops for t in years if value(x1_2[i, j, t]) > 0], columns=["地块名称", "作物名称", "年份", "种植面积/亩"])

# 打印求解状态
print("Problem 1_1 status:", LpStatus[prob1_1.status])
print("Problem 1_2 status:", LpStatus[prob1_2.status])

# 按照要求整理结果
result1_1_formatted = {}
result1_2_formatted = {}
for t in years:
    result1_1_formatted[t] = pd.DataFrame(0, index=plots, columns=crops)
    result1_2_formatted[t] = pd.DataFrame(0, index=plots, columns=crops)

for _, row in result1_1.iterrows():
    plot_name = row["地块名称"]
    crop_name = row["作物名称"]
    year = row["年份"]
    planting_area = row["种植面积/亩"]

    result1_1_formatted[year].loc[plot_name, crop_name] = planting_area

for _, row in result1_2.iterrows():
    plot_name = row["地块名称"]
    crop_name = row["作物名称"]
    year = row["年份"]
    planting_area = row["种植面积/亩"]

    result1_2_formatted[year].loc[plot_name, crop_name] = planting_area
# 保存结果
with pd.ExcelWriter("result1_1.xlsx") as writer:
    for t, df in result1_1_formatted.items():
        df.to_excel(writer, sheet_name=str(t))

with pd.ExcelWriter("result1_2.xlsx") as writer:
    for t, df in result1_2_formatted.items():
        df.to_excel(writer, sheet_name=str(t))

d:\Dev\Anaconda\Lib\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Problem 1_1 status: Infeasible
Problem 1_2 status: Infeasible


C:\Users\jiang\AppData\Local\Temp\ipykernel_31644\96251431.py:86: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '80.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result1_1_formatted[year].loc[plot_name, crop_name] = planting_area
C:\Users\jiang\AppData\Local\Temp\ipykernel_31644\96251431.py:86: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result1_1_formatted[year].loc[plot_name, crop_name] = planting_area
C:\Users\jiang\AppData\Local\Temp\ipykernel_31644\96251431.py:86: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first